In [ ]:
# ModernTCN 전력 소비량 예측 모델
# XGBoost 대신 ModernTCN-short-term 사용

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
import os
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
import random as rn

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# 시드 설정
RANDOM_SEED = 2025
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)


🚀 ModernTCN 전력 소비량 예측 모델 시작!
📱 사용 디바이스: cuda
✅ ModernTCN 모델 및 유틸리티 함수 정의 완료


In [2]:
# 전체 데이터 기간 확인
print("🧪 전체 데이터 기간 확인...")

# 데이터 로드
train = pd.read_csv('data/train.csv')
train = train.rename(columns={'일시': 'date_time'})
train['date_time'] = pd.to_datetime(train['date_time'], format='%Y%m%d %H')

print(f"📅 전체 train 데이터 기간: {train['date_time'].min()} ~ {train['date_time'].max()}")
print(f"📊 전체 train 데이터 수: {len(train)}개")
print(f"📊 총 기간: {(train['date_time'].max() - train['date_time'].min()).days}일")
print(f"📊 총 주차: {(train['date_time'].max() - train['date_time'].min()).days // 7}주")

# 건물 1개의 전체 데이터로 테스트
building_1_data = train[train['건물번호'] == 1].copy()
print(f"\\n📊 건물 1 데이터: {len(building_1_data)}개")
print(f"📅 건물 1 기간: {building_1_data['date_time'].min()} ~ {building_1_data['date_time'].max()}")
print(f"📊 건물 1 총 기간: {(building_1_data['date_time'].max() - building_1_data['date_time'].min()).days}일")
print(f"📊 건물 1 총 주차: {(building_1_data['date_time'].max() - building_1_data['date_time'].min()).days // 7}주")


🧪 전체 데이터 기간 확인...
📅 전체 train 데이터 기간: 2024-06-01 00:00:00 ~ 2024-08-24 23:00:00
📊 전체 train 데이터 수: 204000개
📊 총 기간: 84일
📊 총 주차: 12주
\n📊 건물 1 데이터: 2040개
📅 건물 1 기간: 2024-06-01 00:00:00 ~ 2024-08-24 23:00:00
📊 건물 1 총 기간: 84일
📊 건물 1 총 주차: 12주


In [3]:
def smape(gt, preds):
    """SMAPE (Symmetric Mean Absolute Percentage Error) 계산"""
    gt = np.array(gt)
    preds = np.array(preds)
    v = 2 * abs(preds - gt) / (abs(preds) + abs(gt))
    score = np.mean(v) * 100
    return score
    
def weighted_mse(alpha=1):
    """가중 MSE 손실 함수 (Under-prediction에 더 큰 페널티)"""
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual > 0, -2 * alpha * residual, -2 * residual)
        hess = np.where(residual > 0, 2 * alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

def custom_smape(preds, dtrain):
    """XGBoost용 SMAPE 평가 함수"""
    labels = dtrain.get_label()
    return 'custom_smape', np.mean(2 * abs(preds - labels) / (abs(preds) + abs(labels))) * 100

print("✅ 평가 함수 정의 완료")


✅ 평가 함수 정의 완료


In [4]:
# 데이터 로드
print("📊 데이터 로드 중...")
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
building_info = pd.read_csv('data/building_info.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

print(f"✅ Train 데이터: {train.shape}")
print(f"✅ Test 데이터: {test.shape}")
print(f"✅ Building info: {building_info.shape}")

# 컬럼명 영어로 변경 (작년 수상자 방식)
train = train.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(°C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation',
    '전력소비량(kWh)': 'power_consumption'
})
train.drop('num_date_time', axis=1, inplace=True)

test = test.rename(columns={
    '건물번호': 'building_number',
    '일시': 'date_time',
    '기온(°C)': 'temperature',
    '강수량(mm)': 'rainfall',
    '풍속(m/s)': 'windspeed',
    '습도(%)': 'humidity',
    '일조(hr)': 'sunshine',
    '일사(MJ/m2)': 'solar_radiation'
})
test.drop('num_date_time', axis=1, inplace=True)

building_info = building_info.rename(columns={
    '건물번호': 'building_number',
    '건물유형': 'building_type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar_power_capacity',
    'ESS저장용량(kWh)': 'ess_capacity',
    'PCS용량(kW)': 'pcs_capacity'
})

# 건물 유형 영어로 번역
translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '학교': 'University',
    '백화점': 'Department Store',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    'IDC(전화국)': 'IDC',
    '호텔': 'Hotel'
}
building_info['building_type'] = building_info['building_type'].replace(translation_dict)

# 태양광/ESS 설비 유무 피처 생성
building_info['solar_power_utility'] = np.where(building_info.solar_power_capacity != '-', 1, 0)
building_info['ess_utility'] = np.where(building_info.ess_capacity != '-', 1, 0)

# 건물 정보 병합
train = pd.merge(train, building_info, on='building_number', how='left')
test = pd.merge(test, building_info, on='building_number', how='left')

print("✅ 기본 전처리 완료")


📊 데이터 로드 중...
✅ Train 데이터: (204000, 10)
✅ Test 데이터: (16800, 7)
✅ Building info: (100, 7)
✅ 기본 전처리 완료


In [5]:
# 날짜/시간 변환 및 기본 시간 피처 생성
print("🔧 Feature Engineering 시작...")

train['date_time'] = pd.to_datetime(train['date_time'], format='%Y%m%d %H')
test['date_time'] = pd.to_datetime(test['date_time'], format='%Y%m%d %H')

# 기본 시간 피처
for df in [train, test]:
    df['hour'] = df['date_time'].dt.hour
    df['day'] = df['date_time'].dt.day
    df['month'] = df['date_time'].dt.month
    df['day_of_week'] = df['date_time'].dt.dayofweek

print("✅ 기본 시간 피처 생성 완료")


🔧 Feature Engineering 시작...
✅ 기본 시간 피처 생성 완료


In [6]:
# 일별 온도 통계 피처 생성
def calculate_day_values(dataframe, target_column, output_column, aggregation_func):
    """일별 통계값 계산 함수"""
    result_dict = {}
    grouped_temp = dataframe.groupby(['building_number', 'month', 'day'])[target_column].agg(aggregation_func)
    
    for (building, month, day), value in grouped_temp.items():
        result_dict.setdefault(building, {}).setdefault(month, {})[day] = value
    
    dataframe[output_column] = [
        result_dict.get(row['building_number'], {}).get(row['month'], {}).get(row['day'], None)
        for _, row in dataframe.iterrows()
    ]

# 일별 온도 통계 피처 생성
for df in [train, test]:
    calculate_day_values(df, 'temperature', 'day_max_temperature', 'max')
    calculate_day_values(df, 'temperature', 'day_mean_temperature', 'mean')
    calculate_day_values(df, 'temperature', 'day_min_temperature', 'min')
    df['day_temperature_range'] = df['day_max_temperature'] - df['day_min_temperature']

print("✅ 일별 온도 통계 피처 생성 완료")


✅ 일별 온도 통계 피처 생성 완료


In [7]:
# 이상치 제거 및 추가 피처 생성 (0 제거)
outlier_idx = train.index[train['power_consumption'] == 0].tolist()
print(f"제거할 이상치 개수: {len(outlier_idx)}")
train.drop(index=outlier_idx, inplace=True)
print(f"남은 행 개수: {train.shape[0]}")

제거할 이상치 개수: 68
남은 행 개수: 203932


In [8]:
# 이상치 제거 (Z-score 기반 )
import numpy as np
from scipy import stats

def detect_outliers_zscore_only(train, threshold=4.5):
    """
    Z-score 방법으로만 이상치를 찾아내는 함수
    
    Args:
        train (pd.DataFrame): 전처리된 전체 학습 데이터
        threshold (float): Z-score 임계값 (기본값: 3)
    
    Returns:
        pd.DataFrame: 이상치 데이터
    """
    print(f"�� Z-score 방법으로 이상치 탐지 (임계값: {threshold})")
    print("=" * 60)
    
    outliers_list = []
    
    for building_num in train['building_number'].unique():
        building_data = train[train['building_number'] == building_num]['power_consumption']
        
        # Z-score 계산
        z_scores = np.abs(stats.zscore(building_data))
        
        # 임계값을 초과하는 이상치 찾기
        building_outliers = train[(train['building_number'] == building_num) & 
                                          (z_scores > threshold)]
        
        if not building_outliers.empty:
            outliers_list.append(building_outliers)
    
    outliers_df = pd.concat(outliers_list) if outliers_list else pd.DataFrame()
    
    print(f"📊 발견된 이상치: {len(outliers_df)}개")
    
    if not outliers_df.empty:
        print(f"�� 이상치가 있는 건물 수: {outliers_df['building_number'].nunique()}개")
        print(f"🏢 이상치가 있는 건물 번호: {sorted(outliers_df['building_number'].unique())}")
        
        # 건물별 이상치 개수
        building_counts = outliers_df['building_number'].value_counts()
        print(f"\n📈 건물별 이상치 개수:")
        for building_num, count in building_counts.items():
            building_type = building_info[building_info['building_number'] == building_num]['building_type'].iloc[0]
            print(f"   건물 {building_num} ({building_type}): {count}개")
    else:
        print("✅ 이상치가 발견되지 않았습니다.")
    
    return outliers_df

# Z-score 이상치 탐지 실행
outliers_df = detect_outliers_zscore_only(train, threshold=4.5)

# 이상치 상세 분석
if not outliers_df.empty:
    print(f"\n📊 이상치 상세 분석:")
    print("=" * 60)
    
    for building_num in sorted(outliers_df['building_number'].unique()):
        building_outliers = outliers_df[outliers_df['building_number'] == building_num]
        building_type = building_info[building_info['building_number'] == building_num]['building_type'].iloc[0]
        
        print(f"\n🏢 건물 {building_num} ({building_type}):")
        print(f"   이상치 개수: {len(building_outliers)}개")
        print(f"   최소 전력소비량: {building_outliers['power_consumption'].min():.2f} kWh")
        print(f"   최대 전력소비량: {building_outliers['power_consumption'].max():.2f} kWh")
        print(f"   평균 전력소비량: {building_outliers['power_consumption'].mean():.2f} kWh")
        
        # 전체 건물 데이터와 비교
        building_all = train[train['building_number'] == building_num]
        print(f"   전체 데이터 평균: {building_all['power_consumption'].mean():.2f} kWh")
        print(f"   이상치 비율: {len(building_outliers)/len(building_all)*100:.2f}%")

�� Z-score 방법으로 이상치 탐지 (임계값: 4.5)
📊 발견된 이상치: 68개
�� 이상치가 있는 건물 수: 11개
🏢 이상치가 있는 건물 번호: [23, 30, 41, 43, 52, 64, 67, 72, 76, 81, 99]

📈 건물별 이상치 개수:
   건물 67 (IDC): 48개
   건물 41 (Commercial): 4개
   건물 43 (IDC): 4개
   건물 30 (IDC): 2개
   건물 52 (IDC): 2개
   건물 76 (Commercial): 2개
   건물 99 (Commercial): 2개
   건물 23 (Research Institute): 1개
   건물 64 (IDC): 1개
   건물 72 (Public): 1개
   건물 81 (IDC): 1개

📊 이상치 상세 분석:

🏢 건물 23 (Research Institute):
   이상치 개수: 1개
   최소 전력소비량: 9324.00 kWh
   최대 전력소비량: 9324.00 kWh
   평균 전력소비량: 9324.00 kWh
   전체 데이터 평균: 2339.77 kWh
   이상치 비율: 0.05%

🏢 건물 30 (IDC):
   이상치 개수: 2개
   최소 전력소비량: 2444.40 kWh
   최대 전력소비량: 7374.24 kWh
   평균 전력소비량: 4909.32 kWh
   전체 데이터 평균: 9801.04 kWh
   이상치 비율: 0.10%

🏢 건물 41 (Commercial):
   이상치 개수: 4개
   최소 전력소비량: 246.24 kWh
   최대 전력소비량: 2102.94 kWh
   평균 전력소비량: 1373.76 kWh
   전체 데이터 평균: 2710.89 kWh
   이상치 비율: 0.20%

🏢 건물 43 (IDC):
   이상치 개수: 4개
   최소 전력소비량: 834.60 kWh
   최대 전력소비량: 9765.60 kWh
   평균 전력소비량: 5587.50 kWh
   전체 데이터 평균: 14058.8

In [9]:
def drop_outliers_safe(train, outliers_df):
    """
    안전한 방법으로 이상치 제거 (모든 컬럼 매칭)
    
    Args:
        train (pd.DataFrame): 전체 학습 데이터
        outliers_df (pd.DataFrame): 이상치 데이터
    
    Returns:
        pd.DataFrame: 이상치가 제거된 데이터
    """
    print(f"🔍 안전한 이상치 제거 시작")
    print(f"   원본 데이터 크기: {len(train)}개")
    print(f"   제거할 이상치 개수: {len(outliers_df)}개")
    
    # 정제된 데이터 초기화
    cleaned_data = train.copy()
    
    # 이상치가 있는 건물들만 처리
    outlier_buildings = outliers_df['building_number'].unique()
    
    removed_count = 0
    for building_num in outlier_buildings:
        building_outliers = outliers_df[outliers_df['building_number'] == building_num]
        
        # 해당 건물의 데이터에서 이상치 제거
        building_mask = cleaned_data['building_number'] == building_num
        
        for _, outlier_row in building_outliers.iterrows():
            # 정확히 일치하는 행 찾기
            match_mask = (
                (cleaned_data['building_number'] == outlier_row['building_number']) &
                (cleaned_data['date_time'] == outlier_row['date_time']) &
                (cleaned_data['power_consumption'] == outlier_row['power_consumption'])
            )
            
            # 일치하는 행 제거
            cleaned_data = cleaned_data[~match_mask]
            removed_count += match_mask.sum()
    
    print(f"   제거 후 데이터 크기: {len(cleaned_data)}개")
    print(f"   실제 제거된 행 수: {removed_count}개")
    print(f"   제거된 데이터 비율: {removed_count/len(train)*100:.2f}%")
    
    return cleaned_data

# 안전한 이상치 제거 실행
train = drop_outliers_safe(train, outliers_df)

# 결과 확인
print(f"\n📊 안전한 이상치 제거 결과:")
print(f"   확인: {len(train)}개")

# 정제된 데이터를 새로운 변수에 저장
print(f"\n✅ train 변수에 정제된 데이터가 저장되었습니다.")

🔍 안전한 이상치 제거 시작
   원본 데이터 크기: 203932개
   제거할 이상치 개수: 68개
   제거 후 데이터 크기: 203864개
   실제 제거된 행 수: 68개
   제거된 데이터 비율: 0.03%

📊 안전한 이상치 제거 결과:
   확인: 203864개

✅ train 변수에 정제된 데이터가 저장되었습니다.


In [10]:
# 공휴일(holiday), 주말(weekend), 닫은날(close)로 세분화# 

In [11]:
# 1. 기본 공휴일 및 주말 피처 생성
holi_weekday = ['2024-06-06', '2024-08-15']  # 공휴일 목록
for df in [train, test]:
    # 공휴일 (국가 공휴일)
    df['holiday'] = np.where(
        df.date_time.dt.strftime('%Y-%m-%d').isin(holi_weekday), 1, 0
    )
    
    # 주말 (토요일, 일요일)
    df['weekend'] = np.where(
        df.day_of_week >= 5, 1, 0
    )
    
    # 닫은날 (초기값 0으로 설정, 나중에 건물별 규칙 적용)
    df['close'] = 0


# 주기성 피처 생성 (Cyclical Features)
for df in [train, test]:
    # 시간 주기성
    df['sin_hour'] = np.sin(2 * np.pi * df['hour'] / 23.0)
    df['cos_hour'] = np.cos(2 * np.pi * df['hour'] / 23.0)
    
    # 날짜 주기성
    df['sin_date'] = -np.sin(2 * np.pi * (df['month'] + df['day'] / 31) / 12)
    df['cos_date'] = -np.cos(2 * np.pi * (df['month'] + df['day'] / 31) / 12)
    
    # 월 주기성
    df['sin_month'] = -np.sin(2 * np.pi * df['month'] / 12.0)
    df['cos_month'] = -np.cos(2 * np.pi * df['month'] / 12.0)
    
    # 요일 주기성
    df['sin_dayofweek'] = -np.sin(2 * np.pi * (df['day_of_week'] + 1) / 7.0)
    df['cos_dayofweek'] = -np.cos(2 * np.pi * (df['day_of_week'] + 1) / 7.0)



In [12]:
def apply_specific_building_holidays(df):
    """
    특정 건물들의 휴일 규칙을 적용하는 함수
    - 18번 건물: 매주 일요일 휴무 (close)
    - 27, 40, 59, 63번 건물: 홀수 주 일요일 휴무 (close)
    - 29번 건물: 매달 10일 + 5번째 주 일요일 휴무 (close)
    - 32번 건물: 홀수 주 월요일 휴무 (close)
    """
    print("🔧 특정 건물 휴일 규칙 적용 중... (달력 기준 주차)")
    department_store_buildings = building_info[building_info['building_type'] == 'Department Store']['building_number'].tolist()

    # 매주 일요일 휴무 건물 (18번)
    every_sunday_buildings = [18]

    # 홀수 주 일요일 휴무 건물들 (27, 40, 59, 63)
    odd_week_sunday_buildings = [27, 40, 59, 63]

    # 매달 10일 + 5번째 주 일요일 휴무 건물 (29)
    special_holiday_building = [29]

    # 홀수 주 월요일 휴무 건물 (32)
    odd_week_monday_buildings = [32]

    # 달력 기준 주차 계산 (일요일 시작)
    def get_calendar_week_of_month(date_series):
        """달력 기준 주차 계산 (월의 첫날이 포함된 주를 1주차로)"""
        result = []
        for date in date_series:
            first_day = date.replace(day=1)
            first_day_weekday = first_day.weekday()
            days_to_week_start = (first_day_weekday + 1) % 7  # 일요일 기준
            week_start_of_first = first_day - pd.Timedelta(days=days_to_week_start)
            days_since_first_week_start = (date - week_start_of_first).days
            week_num = (days_since_first_week_start // 7) + 1
            result.append(week_num)
        return result

    # 달력 기준 주차 계산
    df['calendar_week_of_month'] = get_calendar_week_of_month(df['date_time'])

    # 0. 매주 일요일 휴무 (18번 건물) - close에 적용
    for building_num in every_sunday_buildings:
        mask = (df['building_number'] == building_num) & (df['day_of_week'] == 6)
        df.loc[mask, 'close'] = 1
        count = mask.sum()
        print(f"   건물 {building_num}: 매주 일요일 휴일 {count}개 적용")

    # 1. 홀수 주 일요일 휴무 (27, 40, 59, 63번 건물) - close에 적용
    for building_num in odd_week_sunday_buildings:
        mask = (df['building_number'] == building_num) & \
               (df['day_of_week'] == 6) & \
               (df['calendar_week_of_month'] % 2 == 1)
        df.loc[mask, 'close'] = 1
        count = mask.sum()
        print(f"   건물 {building_num}: 홀수 주 일요일 휴일 {count}개 적용")

    # 2. 매달 10일 + 5번째 주 일요일 휴무 (29번 건물) - close에 적용
    for building_num in special_holiday_building:
        mask_10th = (df['building_number'] == building_num) & (df['date_time'].dt.day == 10)
        df.loc[mask_10th, 'close'] = 1
        count_10th = mask_10th.sum()
        mask_5th_sunday = (df['building_number'] == building_num) & \
                          (df['day_of_week'] == 6) & \
                          (df['calendar_week_of_month'] == 5)
        df.loc[mask_5th_sunday, 'close'] = 1
        count_5th_sunday = mask_5th_sunday.sum()
        print(f"   건물 {building_num}: 매달 10일 휴일 {count_10th}개, 5번째 주 일요일 {count_5th_sunday}개 적용")

    # 3. 홀수 주 월요일 휴무 (32번 건물) - close에 적용
    for building_num in odd_week_monday_buildings:
        mask = (df['building_number'] == building_num) & \
               (df['day_of_week'] == 0) & \
               (df['calendar_week_of_month'] % 2 == 1)
        df.loc[mask, 'close'] = 1
        count = mask.sum()
        print(f"   건물 {building_num}: 홀수 주 월요일 휴일 {count}개 적용")

    # 임시 컬럼 제거
    df.drop(['calendar_week_of_month'], axis=1, inplace=True)

    return df

In [13]:

def create_inferred_holidays(df, building_info):
    # 규칙이 없는 백화점 건물 번호 추출
    department_store_buildings = [19,34,45,54,73,74,79,88,95]

    # 백화점 건물에 대해 추정 휴일만 적용
    for building_num in department_store_buildings:
        building_data = df[df['building_number'] == building_num].copy()

        if len(building_data) > 0:
            # 일별 전력소비량 계산
            building_data['date'] = building_data['date_time'].dt.date
            daily_consumption = building_data.groupby('date')['power_consumption'].sum()

            # 전체 평균 기준 임계치 (0.7배)
            threshold = daily_consumption.mean() * 0.7
            inferred_holiday_dates = daily_consumption[daily_consumption < threshold].index

            # 추정 휴일을 해당 건물의 holiday 컬럼에 적용
            for holiday_date in inferred_holiday_dates:
                mask = (df['building_number'] == building_num) & (df['date_time'].dt.date == holiday_date)
                df.loc[mask, 'close'] = 1

    return df

# 휴일 피처 생성
train = create_inferred_holidays(train, building_info)

In [14]:
train = apply_specific_building_holidays(train)
test = apply_specific_building_holidays(test)

🔧 특정 건물 휴일 규칙 적용 중... (달력 기준 주차)
   건물 18: 매주 일요일 휴일 288개 적용
   건물 27: 홀수 주 일요일 휴일 120개 적용
   건물 40: 홀수 주 일요일 휴일 120개 적용
   건물 59: 홀수 주 일요일 휴일 120개 적용
   건물 63: 홀수 주 일요일 휴일 120개 적용
   건물 29: 매달 10일 휴일 72개, 5번째 주 일요일 48개 적용
   건물 32: 홀수 주 월요일 휴일 144개 적용
🔧 특정 건물 휴일 규칙 적용 중... (달력 기준 주차)
   건물 18: 매주 일요일 휴일 24개 적용
   건물 27: 홀수 주 일요일 휴일 24개 적용
   건물 40: 홀수 주 일요일 휴일 24개 적용
   건물 59: 홀수 주 일요일 휴일 24개 적용
   건물 63: 홀수 주 일요일 휴일 24개 적용
   건물 29: 매달 10일 휴일 0개, 5번째 주 일요일 24개 적용
   건물 32: 홀수 주 월요일 휴일 24개 적용


In [15]:
# 특정 건물들과 학교/병원/연구소 건물들을 주말/공휴일에 close 처리하는 함수
def apply_weekend_holiday_close_for_specific_buildings(df, building_info):
    """
    특정 건물들과 학교/병원/연구소 건물들을 주말과 공휴일에 close 처리
    
    Args:
        df: train 또는 test 데이터프레임
        building_info: 건물 정보 데이터프레임
    
    Returns:
        df: close 피처가 업데이트된 데이터프레임
    """
    print("🔧 특정 건물들과 학교/병원/연구소 건물들을 주말/공휴일에 close 처리 중...")
    
    # 특정 건물 번호 리스트
    specific_buildings = [6, 16, 20, 51, 86, 47, 69, 38, 50, 66, 68, 72, 80]
    
    # 학교, 병원, 연구소 건물 번호 추출
    school_hospital_research_buildings = building_info[
        building_info['building_type'].isin(['University', 'Hospital', 'Research Institute'])
    ]['building_number'].tolist()
    
    # 모든 대상 건물 번호
    target_buildings = specific_buildings + school_hospital_research_buildings
    
    print(f"📋 대상 건물 수: {len(target_buildings)}개")
    print(f"   특정 건물: {specific_buildings}")
    print(f"   학교/병원/연구소: {school_hospital_research_buildings}")
    
    # 주말 또는 공휴일인 경우 close = 1로 설정
    for building_num in target_buildings:
        # 해당 건물의 데이터에서 주말 또는 공휴일인 경우
        mask = (df['building_number'] == building_num) & \
               ((df['weekend'] == 1) | (df['holiday'] == 1))
        
        # close 피처를 1로 설정
        df.loc[mask, 'close'] = 1
        
        # 적용된 건수 확인
        applied_count = mask.sum()
        if applied_count > 0:
            building_type = building_info[building_info['building_number'] == building_num]['building_type'].iloc[0]
            print(f"   건물 {building_num} ({building_type}): {applied_count}개 close 처리")
    
    # 전체 적용 결과 확인
    total_close_count = df[df['close'] == 1].shape[0]
    print(f"\\n�� 전체 close 처리 결과:")
    print(f"   총 close 건수: {total_close_count}개")
    
    return df

# 함수 실행
train = apply_weekend_holiday_close_for_specific_buildings(train, building_info)
test = apply_weekend_holiday_close_for_specific_buildings(test, building_info)

print("✅ 특정 건물들과 학교/병원/연구소 건물들의 주말/공휴일 close 처리 완료")

🔧 특정 건물들과 학교/병원/연구소 건물들을 주말/공휴일에 close 처리 중...
📋 대상 건물 수: 41개
   특정 건물: [6, 16, 20, 51, 86, 47, 69, 38, 50, 66, 68, 72, 80]
   학교/병원/연구소: [3, 5, 8, 12, 13, 14, 15, 17, 21, 22, 23, 24, 37, 39, 42, 44, 46, 48, 49, 53, 55, 60, 62, 75, 83, 87, 90, 94]
   건물 6 (Commercial): 648개 close 처리
   건물 16 (Commercial): 648개 close 처리
   건물 20 (Commercial): 648개 close 처리
   건물 51 (Commercial): 648개 close 처리
   건물 86 (Commercial): 648개 close 처리
   건물 47 (Other Buildings): 648개 close 처리
   건물 69 (Other Buildings): 648개 close 처리
   건물 38 (Public): 648개 close 처리
   건물 50 (Public): 648개 close 처리
   건물 66 (Public): 648개 close 처리
   건물 68 (Public): 647개 close 처리
   건물 72 (Public): 646개 close 처리
   건물 80 (Public): 648개 close 처리
   건물 3 (Hospital): 648개 close 처리
   건물 5 (University): 648개 close 처리
   건물 8 (University): 645개 close 처리
   건물 12 (University): 648개 close 처리
   건물 13 (Research Institute): 648개 close 처리
   건물 14 (University): 648개 close 처리
   건물 15 (Research Institute): 648개 close 처리
   건물 17 (Hospit

In [16]:
# 휴일 적용 확인용 
def check_holiday(n,option='holiday'):
    if option == 'holiday':
        holiday_dates_n = train.loc[(train['building_number'] == n) & (train['holiday'] == 1), 'date_time'].dt.date
        unique_holiday_dates_n = holiday_dates_n.drop_duplicates().tolist()
        print("train 데이터 휴일 확인")
        print(f"건물 {n}번  휴일: {len(unique_holiday_dates_n)}일")
        print(f"날짜: {unique_holiday_dates_n[:10]}...")  # 처음 10개만 출력
        holiday_dates_n = test.loc[(test['building_number'] == n) & (test['holiday'] == 1), 'date_time'].dt.date
        unique_holiday_dates_n = holiday_dates_n.drop_duplicates().tolist()
        print("test 데이터 휴일 확인")
        print(f"건물 {n}번  휴일: {len(unique_holiday_dates_n)}일")
        print(f"날짜: {unique_holiday_dates_n[:10]}...")  # 처음 10개만 출력        
    if option == 'weekend':
        weekend_dates_n = train.loc[(train['building_number'] == n) & (train['weekend'] == 1), 'date_time'].dt.date
        unique_weekend_dates_n = weekend_dates_n.drop_duplicates().tolist()
        print("train 데이터 주말 확인")
        print(f"건물 {n}번  주말: {len(unique_weekend_dates_n)}일")
        print(f"날짜: {unique_weekend_dates_n[:10]}...")  # 처음 10개만 출력
        weekend_dates_n = test.loc[(test['building_number'] == n) & (test['weekend'] == 1), 'date_time'].dt.date
        unique_weekend_dates_n = weekend_dates_n.drop_duplicates().tolist()
        print("test 데이터 주말 확인")
        print(f"건물 {n}번  주말: {len(unique_weekend_dates_n)}일")
        print(f"날짜: {unique_weekend_dates_n[:10]}...")  # 처음 10개만 출력
    if option == 'close':
        close_dates_n = train.loc[(train['building_number'] == n) & (train['close'] == 1), 'date_time'].dt.date
        unique_close_dates_n = close_dates_n.drop_duplicates().tolist()
        print("train 데이터 닫은날 확인")
        print(f"건물 {n}번  닫은날: {len(unique_close_dates_n)}일")
        print(f"날짜: {unique_close_dates_n[:10]}...")  # 처음 10개만 출력
        close_dates_n = test.loc[(test['building_number'] == n) & (test['close'] == 1), 'date_time'].dt.date
        unique_close_dates_n = close_dates_n.drop_duplicates().tolist()
        print("test 데이터 닫은날 확인")
        print(f"건물 {n}번  닫은날: {len(unique_close_dates_n)}일")
        print(f"날짜: {unique_close_dates_n[:10]}...")  # 처음 10개만 출력
        

In [17]:
check_holiday(19,'close')

train 데이터 닫은날 확인
건물 19번  닫은날: 3일
날짜: [datetime.date(2024, 6, 10), datetime.date(2024, 7, 8), datetime.date(2024, 8, 19)]...
test 데이터 닫은날 확인
건물 19번  닫은날: 0일
날짜: []...


In [18]:
# 기상 관련 파생 피처 생성
def CDH(xs):
    """Cooling Degree Hours 계산"""
    cumsum = np.cumsum(xs - 26)
    return np.concatenate((cumsum[:11], cumsum[11:] - cumsum[:-11]))

def calculate_and_add_cdh(dataframe):
    """건물별 CDH 계산 및 추가"""
    cdhs = []
    for i in range(1, 101):
        temp = dataframe[dataframe['building_number'] == i]['temperature'].values
        cdh = CDH(temp)
        cdhs.append(cdh)
    return np.concatenate(cdhs)

# CDH, THI, WCT 피처 생성
train['CDH'] = calculate_and_add_cdh(train)
test['CDH'] = calculate_and_add_cdh(test)

# THI (Temperature Humidity Index)
train['THI'] = 9/5 * train['temperature'] - 0.55 * (1 - train['humidity']/100) * (9/5 * train['temperature'] - 26) + 32
test['THI'] = 9/5 * test['temperature'] - 0.55 * (1 - test['humidity']/100) * (9/5 * test['temperature'] - 26) + 32

# WCT (Wind Chill Temperature)
train['WCT'] = 13.12 + 0.6125 * train['temperature'] - 11.37 * (train['windspeed']**0.16) + 0.3965 * (train['windspeed']**0.16) * train['temperature']
test['WCT'] = 13.12 + 0.6125 * test['temperature'] - 11.37 * (test['windspeed']**0.16) + 0.3965 * (test['windspeed']**0.16) * test['temperature']

print("✅ 기상 관련 파생 피처 생성 완료")


✅ 기상 관련 파생 피처 생성 완료


In [19]:
# 전력 소비량 기반 통계 피처 생성 (Target-like Features)
print("📊 전력 소비량 기반 통계 피처 생성 중...")

# 건물별 시간대/요일별 평균 및 표준편차
power_mean = pd.pivot_table(train, values='power_consumption', 
                           index=['building_number', 'hour', 'day_of_week'], 
                           aggfunc=np.mean).reset_index()
power_mean.columns = ['building_number', 'hour', 'day_of_week', 'day_hour_mean']

power_std = pd.pivot_table(train, values='power_consumption', 
                          index=['building_number', 'hour', 'day_of_week'], 
                          aggfunc=np.std).reset_index()
power_std.columns = ['building_number', 'hour', 'day_of_week', 'day_hour_std']

# 건물별 시간대별 평균 및 표준편차
power_hour_mean = pd.pivot_table(train, values='power_consumption', 
                                index=['building_number', 'hour'], 
                                aggfunc=np.mean).reset_index()
power_hour_mean.columns = ['building_number', 'hour', 'hour_mean']

power_hour_std = pd.pivot_table(train, values='power_consumption', 
                               index=['building_number', 'hour'], 
                               aggfunc=np.std).reset_index()
power_hour_std.columns = ['building_number', 'hour', 'hour_std']

# 통계 피처 병합
train = train.merge(power_mean, on=['building_number', 'hour', 'day_of_week'], how='left')
train = train.merge(power_std, on=['building_number', 'hour', 'day_of_week'], how='left')
train = train.merge(power_hour_mean, on=['building_number', 'hour'], how='left')
train = train.merge(power_hour_std, on=['building_number', 'hour'], how='left')

test = test.merge(power_mean, on=['building_number', 'hour', 'day_of_week'], how='left')
test = test.merge(power_std, on=['building_number', 'hour', 'day_of_week'], how='left')
test = test.merge(power_hour_mean, on=['building_number', 'hour'], how='left')
test = test.merge(power_hour_std, on=['building_number', 'hour'], how='left')

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

print("✅ 전력 소비량 기반 통계 피처 생성 완료")
print(f"최종 train 데이터 shape: {train.shape}")
print(f"최종 test 데이터 shape: {test.shape}")


📊 전력 소비량 기반 통계 피처 생성 중...
✅ 전력 소비량 기반 통계 피처 생성 완료
최종 train 데이터 shape: (203864, 43)
최종 test 데이터 shape: (16800, 40)


In [31]:
## 🌟 TimesNet 모델 구현 (Time-Series-Library 기반)

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import math

print("🚀 TimesNet 전력 소비량 예측 모델 구현 시작!")
print("=" * 60)

# 1. TimesNet 핵심 컴포넌트들

class TimesBlock(nn.Module):
    def __init__(self, seq_len, pred_len, top_k, d_model, d_ff, num_kernels=6):
        super(TimesBlock, self).__init__()
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.k = top_k
        
        # FFT를 위한 파라미터
        self.conv = nn.Sequential(
            nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1),
            nn.GELU(),
            nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1),
        )
        
        # 2D 컨볼루션 레이어들
        self.conv2d_layers = nn.ModuleList([
            nn.Conv2d(in_channels=d_model, out_channels=d_model, 
                     kernel_size=(3, 3), padding=(1, 1))
            for _ in range(num_kernels)
        ])
        
        self.norm = nn.LayerNorm(d_model)
        self.activation = nn.GELU()
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len, d_model)
        B, T, N = x.shape
        
        # 1. FFT를 통한 주파수 분석
        x_fft = torch.fft.rfft(x.permute(0, 2, 1), dim=-1)  # (B, N, T//2+1)
        
        # 2. 상위 k개 주파수 선택
        amplitude = torch.abs(x_fft)
        _, top_indices = torch.topk(amplitude.mean(dim=1), self.k, dim=-1)
        
        # 3. 2D 변환 및 처리
        period_list = []
        for i in range(self.k):
            period = T // (top_indices[:, i] + 1)  # 주기 계산
            period = torch.clamp(period, min=1, max=T)
            period_list.append(period)
        
        # 4. 각 주기별로 2D 변환하여 처리
        res = []
        for i in range(self.k):
            period = period_list[i]
            
            # 2D 변환: (batch_size, seq_len, d_model) -> (batch_size, d_model, period, seq_len//period)
            if T % period.max() != 0:
                length = (T // period.max() + 1) * period.max()
                padding = torch.zeros([B, (length - T), N]).to(x.device)
                out = torch.cat([x, padding], dim=1)
            else:
                length = T
                out = x
            
            # Reshape to 2D
            out = out.reshape(B, length // period.max(), period.max(), N)
            out = out.permute(0, 3, 1, 2).contiguous()  # (B, N, length//period, period)
            
            # 2D 컨볼루션 적용
            out = self.conv2d_layers[i](out)
            
            # 다시 1D로 변환
            out = out.permute(0, 2, 3, 1).reshape(B, -1, N)
            out = out[:, :T, :]
            res.append(out)
        
        # 5. 결과 합성
        res = torch.stack(res, dim=-1)
        res = torch.mean(res, dim=-1)
        
        # 6. Residual connection과 정규화
        res = res + x
        res = self.norm(res)
        
        return res

class TimesNet(nn.Module):
    def __init__(self, 
                 seq_len=168,
                 pred_len=1, 
                 enc_in=22,
                 d_model=64,
                 d_ff=128,
                 e_layers=3,
                 top_k=5,
                 num_kernels=6,
                 dropout=0.1):
        super(TimesNet, self).__init__()
        
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.model_type = 'TimesNet'
        
        # 입력 임베딩
        self.enc_embedding = nn.Linear(enc_in, d_model)
        
        # TimesBlock 레이어들
        self.encoder = nn.ModuleList([
            TimesBlock(seq_len, pred_len, top_k, d_model, d_ff, num_kernels)
            for _ in range(e_layers)
        ])
        
        # 정규화 및 드롭아웃
        self.layer_norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
        # 예측 헤드
        self.projection = nn.Linear(d_model, pred_len)
        
        print(f"   TimesNet 초기화: seq_len={seq_len}, pred_len={pred_len}, enc_in={enc_in}")
        print(f"   d_model={d_model}, layers={e_layers}, top_k={top_k}")
        
    def forward(self, x):
        # x shape: (batch_size, seq_len, enc_in)
        
        # 입력 임베딩
        enc_out = self.enc_embedding(x)  # (batch_size, seq_len, d_model)
        enc_out = self.dropout(enc_out)
        
        # TimesBlock 레이어들 통과
        for layer in self.encoder:
            enc_out = layer(enc_out)
        
        # 정규화
        enc_out = self.layer_norm(enc_out)
        
        # 예측: 마지막 시점의 특징을 사용
        output = self.projection(enc_out[:, -1, :])  # (batch_size, pred_len)
        
        return output

# 2. TimesNet용 데이터셋 클래스
class TimesNetDataset(Dataset):
    def __init__(self, data, target, seq_len=168, pred_len=1):
        self.data = data
        self.target = target
        self.seq_len = seq_len
        self.pred_len = pred_len
        
        print(f"   TimesNet 데이터셋 초기화: data shape={data.shape}, seq_len={seq_len}")
        
    def __len__(self):
        return max(0, len(self.data) - self.seq_len - self.pred_len + 1)
    
    def __getitem__(self, idx):
        x = self.data[idx:idx + self.seq_len]  # (seq_len, num_features)
        y = self.target[idx + self.seq_len:idx + self.seq_len + self.pred_len]  # (pred_len,)
        return torch.FloatTensor(x), torch.FloatTensor(y)

# 3. TimesNet 훈련 함수
# 🔧 수정된 train_timesnet 함수 (정확한 SMAPE 계산)

def train_timesnet_fixed(model, train_loader, val_loader, scaler_y, epochs=1000, learning_rate=0.01):
    """수정된 TimesNet 모델 훈련 (정확한 SMAPE 계산)"""
    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)
    
    best_val_loss = float('inf')
    patience = 15
    patience_counter = 0
    
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        # 훈련
        model.train()
        train_loss = 0.0
        train_count = 0
        
        for batch_idx, (batch_x, batch_y) in enumerate(train_loader):
            try:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                
                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y.squeeze())
                loss.backward()
                
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                
                train_loss += loss.item()
                train_count += 1
                
            except Exception as e:
                if batch_idx < 3:
                    print(f"   훈련 배치 {batch_idx} 오류: {e}")
                continue
        
        if train_count == 0:
            break
            
        train_loss /= train_count
        train_losses.append(train_loss)
        
        # 검증
        model.eval()
        val_loss = 0.0
        val_count = 0
        val_predictions_raw = []  # 정규화된 값 (loss 계산용)
        val_targets_raw = []
        val_predictions_denorm = []  # 역정규화된 값 (SMAPE 계산용)
        val_targets_denorm = []
        
        with torch.no_grad():
            for batch_idx, (batch_x, batch_y) in enumerate(val_loader):
                try:
                    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                    
                    outputs = model(batch_x)
                    loss = criterion(outputs, batch_y.squeeze())
                    val_loss += loss.item()
                    val_count += 1
                    
                    # 정규화된 값 저장 (loss 계산용)
                    val_predictions_raw.extend(outputs.cpu().numpy())
                    val_targets_raw.extend(batch_y.cpu().numpy().flatten())
                    
                    # 🎯 역정규화해서 실제 값으로 변환 (SMAPE 계산용)
                    pred_denorm = scaler_y.inverse_transform(outputs.cpu().numpy().reshape(-1, 1)).flatten()
                    true_denorm = scaler_y.inverse_transform(batch_y.cpu().numpy().reshape(-1, 1)).flatten()
                    
                    val_predictions_denorm.extend(pred_denorm)
                    val_targets_denorm.extend(true_denorm)
                    
                except Exception as e:
                    if batch_idx < 3:
                        print(f"   검증 배치 {batch_idx} 오류: {e}")
                    continue
        
        if val_count == 0:
            break
            
        val_loss /= val_count
        val_losses.append(val_loss)
        
        scheduler.step(val_loss)
        
        # Early Stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f"   🛑 조기 종료: 에포크 {epoch+1}")
            break
        
        # 🎯 정확한 SMAPE로 진행 상황 출력
        if (epoch + 1) % 10 == 0:
            if val_predictions_denorm and val_targets_denorm:
                # 실제 값으로 SMAPE 계산
                val_smape = smape(val_targets_denorm, val_predictions_denorm)
                print(f"   📊 에포크 {epoch+1:3d}: 훈련 손실 {train_loss:.6f}, 검증 손실 {val_loss:.6f}, SMAPE {val_smape:.4f}")
            else:
                print(f"   📊 에포크 {epoch+1:3d}: 훈련 손실 {train_loss:.6f}, 검증 손실 {val_loss:.6f}")
    
    return model, train_losses, val_losses

print("✅ 수정된 TimesNet 훈련 함수 정의 완료 (정확한 SMAPE 계산)")

# 4. 피처 설정
print("📊 TimesNet용 피처 설정...")
feature_columns = [
    'temperature', 'windspeed', 'humidity', 'rainfall', 'sunshine', 'solar_radiation',
    'sin_hour', 'cos_hour', 'sin_dayofweek', 'cos_dayofweek', 'sin_month', 'cos_month',
    'total_area', 'cooling_area', 'CDH', 'THI', 'WCT',
    'day_hour_mean', 'hour_mean', 'close', 'weekend', 'holiday'
]

# 사용 가능한 피처만 선택
available_features = [f for f in feature_columns if f in train.columns]
print(f"사용할 피처 ({len(available_features)}개): {available_features}")

NUM_FEATURES = len(available_features)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

# 5. 데이터 분할
train_mask = train['date_time'] < pd.to_datetime('2024-08-17')
val_mask = train['date_time'] >= pd.to_datetime('2024-08-17')

print("✅ TimesNet 모델 및 유틸리티 함수 정의 완료")

🚀 TimesNet 전력 소비량 예측 모델 구현 시작!
✅ 수정된 TimesNet 훈련 함수 정의 완료 (정확한 SMAPE 계산)
📊 TimesNet용 피처 설정...
사용할 피처 (22개): ['temperature', 'windspeed', 'humidity', 'rainfall', 'sunshine', 'solar_radiation', 'sin_hour', 'cos_hour', 'sin_dayofweek', 'cos_dayofweek', 'sin_month', 'cos_month', 'total_area', 'cooling_area', 'CDH', 'THI', 'WCT', 'day_hour_mean', 'hour_mean', 'close', 'weekend', 'holiday']
사용 디바이스: cuda
✅ TimesNet 모델 및 유틸리티 함수 정의 완료


In [32]:
## 🏗️ 수정된 TimesNet 건물 타입별 모델 훈련

print("\n🏗️ 수정된 TimesNet 건물 타입별 모델 훈련...")
print("=" * 60)

building_types = train['building_type'].unique()
timesnet_type_models_fixed = {}
timesnet_type_scores_fixed = {}

for building_type in building_types:
    print(f"\n🔍 건물 유형: {building_type}")
    
    # 해당 유형 데이터 필터링
    type_train_data = train[(train['building_type'] == building_type) & train_mask].sort_values('date_time')
    type_val_data = train[(train['building_type'] == building_type) & val_mask].sort_values('date_time')
    
    if len(type_train_data) < 500 or len(type_val_data) < 100:
        print(f"   ❌ 데이터 부족 (훈련: {len(type_train_data)}, 검증: {len(type_val_data)}), 건너뜀")
        continue
    
    # 피처와 타겟 준비
    X_train = type_train_data[available_features].fillna(0).values
    y_train = type_train_data['power_consumption'].values
    X_val = type_val_data[available_features].fillna(0).values
    y_val = type_val_data['power_consumption'].values
    
    print(f"   📊 데이터 shape: X_train={X_train.shape}, y_train={y_train.shape}")
    
    # 정규화
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_val_scaled = scaler_X.transform(X_val)
    y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_val_scaled = scaler_y.transform(y_val.reshape(-1, 1)).flatten()
    
    # TimesNet 데이터셋 생성
    seq_len = min(168, len(X_train_scaled) // 10)
    train_dataset = TimesNetDataset(X_train_scaled, y_train_scaled, seq_len=seq_len, pred_len=1)
    val_dataset = TimesNetDataset(X_val_scaled, y_val_scaled, seq_len=seq_len, pred_len=1)
    
    if len(train_dataset) == 0 or len(val_dataset) == 0:
        print(f"   ❌ 시퀀스 데이터 부족, 건너뜀")
        continue
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, drop_last=True)
    
    # TimesNet 모델 초기화
    model = TimesNet(
        seq_len=seq_len,
        pred_len=1,
        enc_in=NUM_FEATURES,
        d_model=64,
        d_ff=128,
        e_layers=2,
        top_k=5,
        num_kernels=6,
        dropout=0.1
    ).to(device)
    
    # 🔧 수정된 모델 훈련 (scaler_y 전달)
    model, train_losses, val_losses = train_timesnet_fixed(
        model, train_loader, val_loader, scaler_y, 
        epochs=100, learning_rate=0.001
    )
    
    # 최종 성능 평가
    model.eval()
    val_predictions = []
    val_targets = []
    
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            try:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x)
                
                # 역정규화
                pred_denorm = scaler_y.inverse_transform(outputs.cpu().numpy().reshape(-1, 1)).flatten()
                true_denorm = scaler_y.inverse_transform(batch_y.cpu().numpy().reshape(-1, 1)).flatten()
                
                val_predictions.extend(pred_denorm)
                val_targets.extend(true_denorm)
            except Exception as e:
                print(f"   평가 오류: {e}")
                continue
    
    if val_predictions and val_targets:
        # 최종 SMAPE 계산
        final_smape = smape(val_targets, val_predictions)
        timesnet_type_scores_fixed[building_type] = final_smape
        
        # 모델 저장
        timesnet_type_models_fixed[building_type] = {
            'model': model,
            'scaler_X': scaler_X,
            'scaler_y': scaler_y,
            'seq_len': seq_len
        }
        
        print(f"   ✅ 최종 TimesNet SMAPE: {final_smape:.4f}")
    else:
        print(f"   ❌ 평가 실패")

print(f"\n✅ 수정된 TimesNet 건물 타입별 모델 훈련 완료 ({len(timesnet_type_models_fixed)}개)")

# 성능 비교
if timesnet_type_scores_fixed:
    print("\n📊 수정된 TimesNet 성능 결과:")
    print("-" * 60)
    for building_type in timesnet_type_scores_fixed.keys():
        fixed_score = timesnet_type_scores_fixed[building_type]
        print(f"   {building_type:15s}: SMAPE {fixed_score:7.4f}")


🏗️ 수정된 TimesNet 건물 타입별 모델 훈련...

🔍 건물 유형: Hotel
   📊 데이터 shape: X_train=(18479, 22), y_train=(18479,)
   TimesNet 데이터셋 초기화: data shape=(18479, 22), seq_len=168
   TimesNet 데이터셋 초기화: data shape=(1920, 22), seq_len=168
   TimesNet 초기화: seq_len=168, pred_len=1, enc_in=22
   d_model=64, layers=2, top_k=5
   📊 에포크  10: 훈련 손실 1.005131, 검증 손실 1.104926, SMAPE 68.1131
   📊 에포크  20: 훈련 손실 1.004709, 검증 손실 1.108591, SMAPE 67.6555
   📊 에포크  30: 훈련 손실 1.003748, 검증 손실 1.111856, SMAPE 67.5463
   🛑 조기 종료: 에포크 32
   ✅ 최종 TimesNet SMAPE: 67.6401

🔍 건물 유형: Commercial
   📊 데이터 shape: X_train=(18467, 22), y_train=(18467,)
   TimesNet 데이터셋 초기화: data shape=(18467, 22), seq_len=168
   TimesNet 데이터셋 초기화: data shape=(1920, 22), seq_len=168
   TimesNet 초기화: seq_len=168, pred_len=1, enc_in=22
   d_model=64, layers=2, top_k=5
   📊 에포크  10: 훈련 손실 1.000476, 검증 손실 1.071715, SMAPE 47.0140
   🛑 조기 종료: 에포크 18
   ✅ 최종 TimesNet SMAPE: 46.9929

🔍 건물 유형: Hospital
   📊 데이터 shape: X_train=(16614, 22), y_train=(16614,)
   Time

KeyboardInterrupt: 

In [ ]:
## 🏠 수정된 TimesNet 개별 건물별 모델 훈련

print("\n🏠 수정된 TimesNet 개별 건물별 모델 훈련...")
print("=" * 60)

# 상위 5개 건물만 테스트 (시간 절약)
building_numbers = sorted(train['building_number'].unique())[:5]
timesnet_individual_models_fixed = {}
timesnet_individual_scores_fixed = {}

for building_num in building_numbers:
    print(f"\n🏢 건물 {building_num} 수정된 TimesNet 훈련 중...")
    
    # 해당 건물 데이터 필터링
    building_train_data = train[(train['building_number'] == building_num) & train_mask].sort_values('date_time')
    building_val_data = train[(train['building_number'] == building_num) & val_mask].sort_values('date_time')
    
    if len(building_train_data) < 500 or len(building_val_data) < 50:
        print(f"   ❌ 데이터 부족 (훈련: {len(building_train_data)}, 검증: {len(building_val_data)}), 건너뜀")
        continue
    
    # 피처와 타겟 준비
    X_train = building_train_data[available_features].fillna(0).values
    y_train = building_train_data['power_consumption'].values
    X_val = building_val_data[available_features].fillna(0).values
    y_val = building_val_data['power_consumption'].values
    
    print(f"   📊 데이터 shape: X_train={X_train.shape}, y_train={y_train.shape}")
    
    # 정규화
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    
    X_train_scaled = scaler_X.fit_transform(X_train)
    X_val_scaled = scaler_X.transform(X_val)
    y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_val_scaled = scaler_y.transform(y_val.reshape(-1, 1)).flatten()
    
    # TimesNet 데이터셋 생성
    seq_len = min(168, len(X_train_scaled) // 10)
    train_dataset = TimesNetDataset(X_train_scaled, y_train_scaled, seq_len=seq_len, pred_len=1)
    val_dataset = TimesNetDataset(X_val_scaled, y_val_scaled, seq_len=seq_len, pred_len=1)
    
    if len(train_dataset) == 0 or len(val_dataset) == 0:
        print(f"   ❌ 시퀀스 데이터 부족, 건너뜀")
        continue
    
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, drop_last=True)
    
    # 개별 건물용 더 작은 TimesNet 모델
    model = TimesNet(
        seq_len=seq_len,
        pred_len=1,
        enc_in=NUM_FEATURES,
        d_model=32,  # 더 작게
        d_ff=64,
        e_layers=2,
        top_k=3,     # 더 작게
        num_kernels=4,
        dropout=0.1
    ).to(device)
    
    # 🔧 수정된 모델 훈련
    model, _, _ = train_timesnet_fixed(
        model, train_loader, val_loader, scaler_y, 
        epochs=50, learning_rate=0.001
    )
    
    # 최종 성능 평가
    model.eval()
    val_predictions = []
    val_targets = []
    
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            try:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = model(batch_x)
                
                # 역정규화
                pred_denorm = scaler_y.inverse_transform(outputs.cpu().numpy().reshape(-1, 1)).flatten()
                true_denorm = scaler_y.inverse_transform(batch_y.cpu().numpy().reshape(-1, 1)).flatten()
                
                val_predictions.extend(pred_denorm)
                val_targets.extend(true_denorm)
            except Exception as e:
                print(f"   평가 오류: {e}")
                continue
    
    if val_predictions and val_targets:
        # 최종 SMAPE 계산
        final_smape = smape(val_targets, val_predictions)
        timesnet_individual_scores_fixed[building_num] = final_smape
        
        # 모델 저장
        timesnet_individual_models_fixed[building_num] = {
            'model': model,
            'scaler_X': scaler_X,
            'scaler_y': scaler_y,
            'seq_len': seq_len
        }
        
        print(f"   ✅ 최종 TimesNet SMAPE: {final_smape:.4f}")
    else:
        print(f"   ❌ 평가 실패")

print(f"\n✅ 수정된 TimesNet 개별 건물 모델 훈련 완료 ({len(timesnet_individual_models_fixed)}개)")

# 성능 비교
if timesnet_individual_scores_fixed:
    print("\n📊 수정된 TimesNet 개별 건물 성능:")
    print("-" * 60)
    for building_num in timesnet_individual_scores_fixed.keys():
        fixed_score = timesnet_individual_scores_fixed[building_num]
        print(f"   건물 {building_num:2d}: SMAPE {fixed_score:7.4f}")

print("\n🎉 수정된 TimesNet 훈련 코드 적용 완료!")
print("이제 훈련 중 SMAPE와 최종 SMAPE가 일치합니다! ✅")

In [ ]:
## 🔮 TimesNet 테스트 예측 및 결과 저장

print("\n🔮 TimesNet 테스트 예측 및 결과 저장...")
print("=" * 50)

def predict_with_timesnet(model_dict, test_data, feature_columns):
    """TimesNet 모델로 테스트 데이터 예측"""
    model = model_dict['model']
    scaler_X = model_dict['scaler_X']
    scaler_y = model_dict['scaler_y']
    seq_len = model_dict['seq_len']
    
    # 피처 준비
    X_test = test_data[feature_columns].fillna(0).values
    X_test_scaled = scaler_X.transform(X_test)
    predictions = []
    
    if len(X_test_scaled) >= seq_len:
        # 슬라이딩 윈도우로 예측
        for i in range(len(X_test_scaled)):
            try:
                if i < seq_len:
                    seq_data = X_test_scaled[:seq_len]
                else:
                    seq_data = X_test_scaled[i-seq_len:i]
                
                seq_tensor = torch.FloatTensor(seq_data).unsqueeze(0).to(device)
                
                model.eval()
                with torch.no_grad():
                    pred_scaled = model(seq_tensor)
                    pred_denorm = scaler_y.inverse_transform(pred_scaled.cpu().numpy().reshape(-1, 1)).flatten()
                    predictions.extend(pred_denorm)
                    
            except Exception as e:
                print(f"   예측 오류 (인덱스 {i}): {e}")
                # 평균값으로 대체
                mean_pred = scaler_y.inverse_transform([[0]])[0][0]
                predictions.append(mean_pred)
    else:
        # 데이터가 부족한 경우
        mean_pred = scaler_y.inverse_transform([[0]])[0][0]
        predictions = [mean_pred] * len(X_test_scaled)
    
    return predictions

# TimesNet 건물 타입별 예측
timesnet_type_predictions = {}
for building_type in timesnet_type_models.keys():
    print(f"   TimesNet 건물 유형 {building_type} 예측 중...")
    
    type_test_data = test[test['building_type'] == building_type].sort_values(['building_number', 'date_time'])
    
    for building_num in type_test_data['building_number'].unique():
        building_test_data = type_test_data[type_test_data['building_number'] == building_num]
        
        preds = predict_with_timesnet(
            timesnet_type_models[building_type], 
            building_test_data, 
            available_features
        )
        
        timesnet_type_predictions[building_num] = preds

# TimesNet 개별 건물별 예측
timesnet_individual_predictions = {}
for building_num in timesnet_individual_models.keys():
    print(f"   TimesNet 건물 {building_num} 예측 중...")
    
    building_test_data = test[test['building_number'] == building_num].sort_values('date_time')
    
    preds = predict_with_timesnet(
        timesnet_individual_models[building_num], 
        building_test_data, 
        available_features
    )
    
    timesnet_individual_predictions[building_num] = preds

# CSV 파일 저장
print("\n💾 TimesNet 예측 결과 CSV 저장...")

# TimesNet 타입별 결과
timesnet_type_submission = sample_submission.copy()
for building_num, preds in timesnet_type_predictions.items():
    mask = timesnet_type_submission['building_number'] == building_num
    if len(preds) == mask.sum():
        timesnet_type_submission.loc[mask, 'answer'] = preds

timesnet_type_submission.to_csv('timesnet_type_model_submission.csv', index=False)
print("✅ TimesNet 타입별 예측 저장: timesnet_type_model_submission.csv")

# TimesNet 개별 건물 결과
timesnet_individual_submission = sample_submission.copy()
for building_num, preds in timesnet_individual_predictions.items():
    mask = timesnet_individual_submission['building_number'] == building_num
    if len(preds) == mask.sum():
        timesnet_individual_submission.loc[mask, 'answer'] = preds

timesnet_individual_submission.to_csv('timesnet_individual_model_submission.csv', index=False)
print("✅ TimesNet 개별 건물 예측 저장: timesnet_individual_model_submission.csv")

# TimesNet 앙상블 (개별 7 : 타입 3)
print("\n🎯 TimesNet 앙상블 예측...")
timesnet_ensemble_submission = sample_submission.copy()

for building_num in timesnet_ensemble_submission['building_number'].unique():
    mask = timesnet_ensemble_submission['building_number'] == building_num
    
    individual_pred = timesnet_individual_predictions.get(building_num)
    type_pred = timesnet_type_predictions.get(building_num)
    
    if individual_pred is not None and type_pred is not None:
        if len(individual_pred) == len(type_pred) == mask.sum():
            ensemble_pred = [0.7 * ind + 0.3 * typ for ind, typ in zip(individual_pred, type_pred)]
            timesnet_ensemble_submission.loc[mask, 'answer'] = ensemble_pred
    elif individual_pred is not None:
        if len(individual_pred) == mask.sum():
            timesnet_ensemble_submission.loc[mask, 'answer'] = individual_pred
    elif type_pred is not None:
        if len(type_pred) == mask.sum():
            timesnet_ensemble_submission.loc[mask, 'answer'] = type_pred

timesnet_ensemble_submission.to_csv('timesnet_ensemble_submission.csv', index=False)
print("✅ TimesNet 앙상블 예측 저장: timesnet_ensemble_submission.csv")

# 최종 결과 요약
print("\n📊 TimesNet 모델 결과 요약")
print("=" * 50)
print(f"사용된 피처 수: {NUM_FEATURES}개")
print(f"TimesNet 타입별 모델 수: {len(timesnet_type_models)}개")
print(f"TimesNet 개별 건물 모델 수: {len(timesnet_individual_models)}개")

if timesnet_type_scores:
    avg_timesnet_type_smape = np.mean(list(timesnet_type_scores.values()))
    print(f"TimesNet 타입별 평균 SMAPE: {avg_timesnet_type_smape:.4f}")

if timesnet_individual_scores:
    avg_timesnet_individual_smape = np.mean(list(timesnet_individual_scores.values()))
    print(f"TimesNet 개별 건물 평균 SMAPE: {avg_timesnet_individual_smape:.4f}")

print("\n📁 저장된 TimesNet 결과 파일:")
print("- timesnet_type_model_submission.csv")
print("- timesnet_individual_model_submission.csv")
print("- timesnet_ensemble_submission.csv")

print("\n🎉 TimesNet 전력 소비량 예측 모델 구현 완료!")